In [2]:
#!pip install -Uqq fastbook
from fastbook import *
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
from fastai.tabular.all import *
#import fastbook
#fastbook.
setup_book()

In [3]:
import pandas as pd
df = pd.read_csv('..\\data\\election_results\\1976-2020-senate.csv', 
                 skiprows=lambda x: 1<=x<=1631, # to start the year 2000 because party politics weird before then
                 # omit data after 2012 because don't have CF scores after that 
                 skipfooter= 768, # last 2012 row is row2861, there are 3629 rows so need to skip the bottom 768 rows
                 encoding='latin1')

C:\Users\bbdil\AppData\Local\Temp\ipykernel_34324\2380158469.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('..\\data\\election_results\\1976-2020-senate.csv',


In [7]:
df

,year,state,state_po,state_fips,state_cen,state_ic,office,district,stage,special,candidate,party_detailed,writein,mode,candidatevotes,totalvotes,unofficial,version,party_simplified
0,2000,ARIZONA,AZ,4,86,61,US SENATE,statewide,gen,False,WILLIAM TOEL,INDEPENDENT,False,total,109230,1397076,False,20210114,OTHER
1,2000,ARIZONA,AZ,4,86,61,US SENATE,statewide,gen,False,VANCE HANSEN,GREEN,False,total,108926,1397076,False,20210114,OTHER
2,2000,ARIZONA,AZ,4,86,61,US SENATE,statewide,gen,False,JON KYL,REPUBLICAN,False,total,1108196,1397076,False,20210114,REPUBLICAN
3,2000,ARIZONA,AZ,4,86,61,US SENATE,statewide,gen,False,BARRY J. HESS II,LIBERTARIAN,False,total,70724,1397076,False,20210114,LIBERTARIAN
4,2000,CALIFORNIA,CA,6,93,71,US SENATE,statewide,gen,False,"JOSE LUIS ""JOE"" CAMAHORT",REFORM,False,total,96552,10623608,False,20210114,OTHER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1225,2012,WYOMING,WY,56,83,68,US SENATE,statewide,gen,False,JOEL OTTO,COUNTRY PARTY,False,total,6176,250700,False,20210114,OTHER
1226,2012,WYOMING,WY,56,83,68,US SENATE,statewide,gen,False,JOHN BARRASSO,REPUBLICAN,False,total,185250,250700,False,20210114,REPUBLICAN
1227,2012,WYOMING,WY,56,83,68,US SENATE,statewide,gen,False,TIM CHESNUT,DEMOCRAT,False,total,53019,250700,False,20210114,DEMOCRAT
1228,2012,WYOMING,WY,56,83,68,US SENATE,statewide,gen,False,OVER VOTE,NaN,False,total,91,250700,False,20210114,OTHER


In [4]:
df = df[(df.special==False) & (df.writein==False) & (df.unofficial==False)]
df
df = df.loc[:,['year','state_po','candidate','candidatevotes','party_simplified','state']]
df
df = df[(df.party_simplified=='REPUBLICAN') | (df.party_simplified=='DEMOCRAT')]
df.head(10)

to_del = []
i = 0
while i<=df.shape[0]-3: # gives indexOB error when I try to go to the end. Works for now because I know (manually) that last one that needs to be deleted is within this range
    start = i # temp variable so we can get length of block later
    count = 0
    curr_state = df.iloc[i].state_po
    #from IPython.core.debugger import Pdb; Pdb().set_trace()
    while df.iloc[i].state_po==curr_state:
        count+=1
        i+=1
    if count == 2:
        pass#i+=1 # move onto next state
    else:
        print(start)
        to_del.extend(range(start,i)) # utilize non-inclusive upper bound because at this point, 
                                      # the next state has already been moved onto by incrementing i 
                                      # in the inner while loop above
to_del
#df

# because .drop() uses an iterable of index LABELS, must do:
to_drop = [df.index[i] for i in to_del]
to_drop

elections = df.drop(to_drop)
elections

0
83
86
96
99
102
129
154
165
216
257
272
277
391
398
455


,year,state_po,candidate,candidatevotes,party_simplified,state
6,2000,CA,TOM CAMPBELL,3886853,REPUBLICAN,CALIFORNIA
7,2000,CA,DIANNE FEINSTEIN,5932522,DEMOCRAT,CALIFORNIA
11,2000,CT,JOSEPH I. LIEBERMAN,828902,DEMOCRAT,CONNECTICUT
13,2000,CT,PHIL GIORDANO,448077,REPUBLICAN,CONNECTICUT
15,2000,DE,THOMAS R. CARPER,181566,DEMOCRAT,DELAWARE
...,...,...,...,...,...,...
1216,2012,WV,JOHN R. RAESE,240787,REPUBLICAN,WEST VIRGINIA
1220,2012,WI,TAMMY BALDWIN,1547104,DEMOCRAT,WISCONSIN
1221,2012,WI,TOMMY G. THOMPSON,1380126,REPUBLICAN,WISCONSIN
1226,2012,WY,JOHN BARRASSO,185250,REPUBLICAN,WYOMING


In [5]:
elections.insert(3,'Dem_candidate',[None for x in range(elections.shape[0])])
elections.insert(4,'Repub_candidate',[None for x in range(elections.shape[0])])
elections.insert(5,'Dem_votes',[None for x in range(elections.shape[0])])
elections.insert(6,'Repub_votes',[None for x in range(elections.shape[0])])

In [6]:
i = 1 # because going to look at previous row
while i<=elections.shape[0]:
    if elections.iloc[i,8]=='DEMOCRAT': # party_simplified
        elections.iloc[i,3]=elections.iloc[i,2] # Dem_candidate, candidate
        elections.iloc[i,4]=elections.iloc[i-1,2] # Repub_candidate, candidate
        elections.iloc[i,5]=elections.iloc[i,7] # Dem_votes, candidatevotes
        elections.iloc[i,6]=elections.iloc[i-1,7] # Repub_votes, candidatevotes
    elif elections.iloc[i,8]=='REPUBLICAN':
        elections.iloc[i,3]=elections.iloc[i-1,2] # Dem_candidate, candidate
        elections.iloc[i,4]=elections.iloc[i,2] # Repub_candidate, candidate
        elections.iloc[i,5]=elections.iloc[i-1,7] # Dem_votes, candidatevotes
        elections.iloc[i,6]=elections.iloc[i,7] # Repub_votes, candidatevotes
    i+=2

In [7]:
elections = elections.iloc[1::2]

In [12]:
elections # keep full state name because PVIs spreadsheet doesnt have abbreviations

,year,state_po,candidate,Dem_candidate,Repub_candidate,Dem_votes,Repub_votes,candidatevotes,party_simplified,state
7,2000,CA,DIANNE FEINSTEIN,DIANNE FEINSTEIN,TOM CAMPBELL,5932522,3886853,5932522,DEMOCRAT,CALIFORNIA
13,2000,CT,PHIL GIORDANO,JOSEPH I. LIEBERMAN,PHIL GIORDANO,828902,448077,448077,REPUBLICAN,CONNECTICUT
18,2000,DE,"WILLIAM V. ROTH, JR.",THOMAS R. CARPER,"WILLIAM V. ROTH, JR.",181566,142891,142891,REPUBLICAN,DELAWARE
22,2000,FL,BILL MCCOLLUM,BILL NELSON,BILL MCCOLLUM,2989487,2705348,2705348,REPUBLICAN,FLORIDA
38,2000,HI,DANIEL K. AKAKA,DANIEL K. AKAKA,JOHN CARROLL,251215,84701,251215,DEMOCRAT,HAWAII
...,...,...,...,...,...,...,...,...,...,...
1211,2012,VA,TIMOTHY M. KAINE,TIMOTHY M. KAINE,GEORGE F. ALLEN,2010067,1785542,2010067,DEMOCRAT,VIRGINIA
1213,2012,WA,MICHAEL BAUMGARTNER,MARIA CANTWELL,MICHAEL BAUMGARTNER,1855493,1213924,1213924,REPUBLICAN,WASHINGTON
1216,2012,WV,JOHN R. RAESE,JOE MANCHIN III,JOHN R. RAESE,399898,240787,240787,REPUBLICAN,WEST VIRGINIA
1221,2012,WI,TOMMY G. THOMPSON,TAMMY BALDWIN,TOMMY G. THOMPSON,1547104,1380126,1380126,REPUBLICAN,WISCONSIN


In [8]:
elections.insert(8,'Repub%of_two-party_vote', elections.Repub_votes/(elections.Dem_votes+elections.Repub_votes))

In [14]:
elections

,year,state_po,candidate,Dem_candidate,Repub_candidate,Dem_votes,Repub_votes,candidatevotes,Repub%of_two-party_vote,party_simplified,state
7,2000,CA,DIANNE FEINSTEIN,DIANNE FEINSTEIN,TOM CAMPBELL,5932522,3886853,5932522,0.395835,DEMOCRAT,CALIFORNIA
13,2000,CT,PHIL GIORDANO,JOSEPH I. LIEBERMAN,PHIL GIORDANO,828902,448077,448077,0.350888,REPUBLICAN,CONNECTICUT
18,2000,DE,"WILLIAM V. ROTH, JR.",THOMAS R. CARPER,"WILLIAM V. ROTH, JR.",181566,142891,142891,0.4404,REPUBLICAN,DELAWARE
22,2000,FL,BILL MCCOLLUM,BILL NELSON,BILL MCCOLLUM,2989487,2705348,2705348,0.475053,REPUBLICAN,FLORIDA
38,2000,HI,DANIEL K. AKAKA,DANIEL K. AKAKA,JOHN CARROLL,251215,84701,251215,0.252149,DEMOCRAT,HAWAII
...,...,...,...,...,...,...,...,...,...,...,...
1211,2012,VA,TIMOTHY M. KAINE,TIMOTHY M. KAINE,GEORGE F. ALLEN,2010067,1785542,2010067,0.470423,DEMOCRAT,VIRGINIA
1213,2012,WA,MICHAEL BAUMGARTNER,MARIA CANTWELL,MICHAEL BAUMGARTNER,1855493,1213924,1213924,0.39549,REPUBLICAN,WASHINGTON
1216,2012,WV,JOHN R. RAESE,JOE MANCHIN III,JOHN R. RAESE,399898,240787,240787,0.375827,REPUBLICAN,WEST VIRGINIA
1221,2012,WI,TOMMY G. THOMPSON,TAMMY BALDWIN,TOMMY G. THOMPSON,1547104,1380126,1380126,0.471478,REPUBLICAN,WISCONSIN


In [9]:
elections = elections.drop(['candidate','Dem_votes','Repub_votes','candidatevotes','party_simplified'], axis=1) 

In [16]:
elections

,year,state_po,Dem_candidate,Repub_candidate,Repub%of_two-party_vote,state
7,2000,CA,DIANNE FEINSTEIN,TOM CAMPBELL,0.395835,CALIFORNIA
13,2000,CT,JOSEPH I. LIEBERMAN,PHIL GIORDANO,0.350888,CONNECTICUT
18,2000,DE,THOMAS R. CARPER,"WILLIAM V. ROTH, JR.",0.4404,DELAWARE
22,2000,FL,BILL NELSON,BILL MCCOLLUM,0.475053,FLORIDA
38,2000,HI,DANIEL K. AKAKA,JOHN CARROLL,0.252149,HAWAII
...,...,...,...,...,...,...
1211,2012,VA,TIMOTHY M. KAINE,GEORGE F. ALLEN,0.470423,VIRGINIA
1213,2012,WA,MARIA CANTWELL,MICHAEL BAUMGARTNER,0.39549,WASHINGTON
1216,2012,WV,JOE MANCHIN III,JOHN R. RAESE,0.375827,WEST VIRGINIA
1221,2012,WI,TAMMY BALDWIN,TOMMY G. THOMPSON,0.471478,WISCONSIN


<font size=8>adding cfscores</font>

In [10]:
raw_cfscores = pd.read_csv('..\\data\\candidate_ideology\\candidate_cfscores_st_fed_1979_2012.csv')

C:\Users\bbdil\AppData\Local\Temp\ipykernel_34324\3681320154.py:1: DtypeWarning: Columns (3,4,10,13,14,41,42,43,44,45,46,63,66,67,68,69,70,71,72,73,74,75,77) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_cfscores = pd.read_csv('..\\data\\candidate_ideology\\candidate_cfscores_st_fed_1979_2012.csv')


In [11]:
cfscores = raw_cfscores.loc[(raw_cfscores.cycle<=2012)&(raw_cfscores.cycle>=2000),['cycle','Name','State','Incum.Chall','cfscore',
                                                            'cfscores.dyn','seat','Party']]

In [12]:
def extract_last_name(name: str) -> str:
    splt = name.split(' ')
    if splt[-1] in ['III','IV','JR','JR.','SR','SR.']:
        final = splt[-2].replace(',', '') # in case comma after last name before JR./III
        return final
    else:
        return splt[-1]

In [13]:
def find_cfscore(party,name,year,state):
    last_name = extract_last_name(name)
    matches = cfscores[(cfscores.Name.str.contains(last_name,regex=True))&(cfscores.cycle==year)
                       &(cfscores.State==state)&(cfscores.Party.astype(str)==party)&(cfscores.seat=='federal:senate')]
    if len(matches)==1:
        return matches.iloc[0,4]
    else:
        return None
    
elections['Dem_cfscore'] = elections.apply(lambda x: find_cfscore('100', x.Dem_candidate, # '100' means Dem
                                                                 x.year, x.state_po), axis=1)
elections['Repub_cfscore'] = elections.apply(lambda x: find_cfscore('200', x.Repub_candidate, # '200' means Repub
                                                                 x.year, x.state_po), axis=1)

In [21]:
elections

,year,state_po,Dem_candidate,Repub_candidate,Repub%of_two-party_vote,state,Dem_cfscore,Repub_cfscore
7,2000,CA,DIANNE FEINSTEIN,TOM CAMPBELL,0.395835,CALIFORNIA,-0.949979,0.570457
13,2000,CT,JOSEPH I. LIEBERMAN,PHIL GIORDANO,0.350888,CONNECTICUT,-0.540546,0.159063
18,2000,DE,THOMAS R. CARPER,"WILLIAM V. ROTH, JR.",0.4404,DELAWARE,-0.600173,0.644361
22,2000,FL,BILL NELSON,BILL MCCOLLUM,0.475053,FLORIDA,-0.619222,0.863845
38,2000,HI,DANIEL K. AKAKA,JOHN CARROLL,0.252149,HAWAII,-0.781409,0.684819
...,...,...,...,...,...,...,...,...
1211,2012,VA,TIMOTHY M. KAINE,GEORGE F. ALLEN,0.470423,VIRGINIA,-1.084158,0.966839
1213,2012,WA,MARIA CANTWELL,MICHAEL BAUMGARTNER,0.39549,WASHINGTON,-1.138804,1.171569
1216,2012,WV,JOE MANCHIN III,JOHN R. RAESE,0.375827,WEST VIRGINIA,-0.189099,1.034782
1221,2012,WI,TAMMY BALDWIN,TOMMY G. THOMPSON,0.471478,WISCONSIN,-1.606083,1.016576


<font size=10>adding PVIs</font>

In [14]:
import pandas as pd
pvis = pd.read_csv('..\\data\\state_partisan_lean\\US Presidential Results & PVIs by State 1828-2012 - 2-Party US Pres Results & PVIs.csv')
pvis

,Year,2012,Unnamed: 2,Unnamed: 3,Unnamed: 4,2008,Unnamed: 6,Unnamed: 7,Unnamed: 8,2004,Unnamed: 10,Unnamed: 11,Unnamed: 12,2000,Unnamed: 14,Unnamed: 15,Unnamed: 16,1996,Unnamed: 18,Unnamed: 19,Unnamed: 20,1992,Unnamed: 22,Unnamed: 23,Unnamed: 24,1988,Unnamed: 26,Unnamed: 27,Unnamed: 28,1984,Unnamed: 30,Unnamed: 31,Unnamed: 32,1980,Unnamed: 34,Unnamed: 35,Unnamed: 36,1976,Unnamed: 38,Unnamed: 39,Unnamed: 40,1972,Unnamed: 42,Unnamed: 43,Unnamed: 44,1964,Unnamed: 46,Unnamed: 47,Unnamed: 48,1960,Unnamed: 50,Unnamed: 51,Unnamed: 52,1956,Unnamed: 54,Unnamed: 55,Unnamed: 56,1952,Unnamed: 58,Unnamed: 59,Unnamed: 60,1944,Unnamed: 62,Unnamed: 63,Unnamed: 64,1940,Unnamed: 66,Unnamed: 67,Unnamed: 68,1936,Unnamed: 70,Unnamed: 71,Unnamed: 72,1932,Unnamed: 74,Unnamed: 75,Unnamed: 76,1928,Unnamed: 78,Unnamed: 79,Unnamed: 80,1920,Unnamed: 82,Unnamed: 83,Unnamed: 84,1916,Unnamed: 86,Unnamed: 87,Unnamed: 88,1908,Unnamed: 90,Unnamed: 91,Unnamed: 92,1904,Unnamed: 94,Unnamed: 95,Unnamed: 96,1900,Unnamed: 98,Unnamed: 99,Unnamed: 100,1896,Unnamed: 102,Unnamed: 103,Unnamed: 104,1888,Unnamed: 106,Unnamed: 107,Unnamed: 108,1884,Unnamed: 110,Unnamed: 111,Unnamed: 112,1880,Unnamed: 114,Unnamed: 115,Unnamed: 116,1876,Unnamed: 118,Unnamed: 119,Unnamed: 120,1872,Unnamed: 122,Unnamed: 123,Unnamed: 124,1868,Unnamed: 126,Unnamed: 127,Unnamed: 128,1864,Unnamed: 130,Unnamed: 131,Unnamed: 132,1852,Unnamed: 134,Unnamed: 135,Unnamed: 136,1848,Unnamed: 138,Unnamed: 139,Unnamed: 140,1844,Unnamed: 142,Unnamed: 143,Unnamed: 144,1840,Unnamed: 146,Unnamed: 147,Unnamed: 148,1836,Unnamed: 150,Unnamed: 151,Unnamed: 152,1832,Unnamed: 154,Unnamed: 155,Unnamed: 156,1828,Unnamed: 158,Unnamed: 159,Unnamed: 160,Unnamed: 161,2012.1,Unnamed: 163,Unnamed: 164,2008.1,Unnamed: 166,Unnamed: 167,2004.1,Unnamed: 169,Unnamed: 170,2000.1,Unnamed: 172,Unnamed: 173,Unnamed: 174,1996.1,Unnamed: 176,Unnamed: 177,Unnamed: 178,1992.1,Unnamed: 180,Unnamed: 181,Unnamed: 182,1988.1,Unnamed: 184,Unnamed: 185,1984.1,Unnamed: 187,Unnamed: 188,1980.1,Unnamed: 190,Unnamed: 191,Unnamed: 192,1976.1,Unnamed: 194,Unnamed: 195,1972.1,Unnamed: 197,Unnamed: 198,1968,Unnamed: 200,Unnamed: 201,Unnamed: 202,1964.1,Unnamed: 204,Unnamed: 205,1960.1,Unnamed: 207,Unnamed: 208,Unnamed: 209,1956.1,Unnamed: 211,Unnamed: 212,Unnamed: 213,1952.1,Unnamed: 215,Unnamed: 216,1948,Unnamed: 218,Unnamed: 219,Unnamed: 220,Unnamed: 221,1944.1,Unnamed: 223,Unnamed: 224,1940.1,Unnamed: 226,Unnamed: 227,1936.1,Unnamed: 229,Unnamed: 230,1932.1,Unnamed: 232,Unnamed: 233,Unnamed: 234,1928.1,Unnamed: 236,Unnamed: 237,1924,Unnamed: 239,Unnamed: 240,Unnamed: 241,1920.1,Unnamed: 243,Unnamed: 244,Unnamed: 245,1916.1,Unnamed: 247,Unnamed: 248,Unnamed: 249,1912,Unnamed: 251,Unnamed: 252,Unnamed: 253,Unnamed: 254,1908.1,Unnamed: 256,Unnamed: 257,Unnamed: 258,1904.1,Unnamed: 260,Unnamed: 261,Unnamed: 262,1900.1,Unnamed: 264,Unnamed: 265,1896.1,Unnamed: 267,Unnamed: 268,1892,Unnamed: 270,Unnamed: 271,Unnamed: 272,1888.1,Unnamed: 274,Unnamed: 275,1884.1,Unnamed: 277,Unnamed: 278,1880.1,Unnamed: 280,Unnamed: 281,Unnamed: 282,1876.1,Unnamed: 284,Unnamed: 285,1872.1,Unnamed: 287,Unnamed: 288,1868.1,Unnamed: 290,Unnamed: 291,1864.1,Unnamed: 293,Unnamed: 294,1860,Unnamed: 296,Unnamed: 297,Unnamed: 298,Unnamed: 299,1856,Unnamed: 301,Unnamed: 302,Unnamed: 303,1852.1,Unnamed: 305,Unnamed: 306,Unnamed: 307,1848.1,Unnamed: 309,Unnamed: 310,Unnamed: 311,1844.1,Unnamed: 313,Unnamed: 314,Unnamed: 315,1840.1,Unnamed: 317,Unnamed: 318,1836.1,Unnamed: 320,Unnamed: 321,1832.1,Unnamed: 323,Unnamed: 324,Unnamed: 325,1828.1,Unnamed: 327,Unnamed: 328,Unnamed: 329,PVI,PVI.1,PVI.2,PVI.3,PVI.4,PVI.5,PVI.6,PVI.7,PVI.8,PVI.9,PVI.10,PVI.11,PVI.12,PVI.13,PVI.14,PVI.15,PVI.16,PVI.17,PVI.18,PVI.19,PVI.20,PVI.21,PVI.22,PVI.23,PVI.24,PVI.25,PVI.26,PVI.27,PVI.28,PVI.29,PVI.30,PVI.31,PVI.32,PVI.33,PVI.34,PVI.35,PVI.36,PVI.37,PVI.38,PVI.39,PVI.40,PVI.41,PVI.42,PVI.43,PVI.44
0,State,"Obama, Barack - Democratic","Romney, Mitt - Republican",PVI,NaN,"Obama, Barack - Democratic","McCain,

In [15]:
pvis = pvis.iloc[:,0:17] # get just 2000-2012

# keep only the rows for each state not nationwide
# and only the column about PVI, not presidential performance
pvis = pvis.iloc[2:,[0,3,4,7,8,11,12,15,16]]

In [16]:
pvis = pvis.rename(columns={'Year':'State',
                     'Unnamed: 3':'2012 PVI direction',
                     'Unnamed: 4':'2012 PVI magnitude',
                     'Unnamed: 7':'2008 PVI direction',
                     'Unnamed: 8':'2008 PVI magnitude',
                     'Unnamed: 11':'2004 PVI direction',
                     'Unnamed: 12':'2004 PVI magnitude',
                     'Unnamed: 15':'2000 PVI direction',
                     'Unnamed: 16':'2000 PVI magnitude',
                    }
           )
pvis

,State,2012 PVI direction,2012 PVI magnitude,2008 PVI direction,2008 PVI magnitude,2004 PVI direction,2004 PVI magnitude,2000 PVI direction,2000 PVI magnitude
2,Alabama,R+,13.0,R+,15.0,R+,12.0,R+,8.0
3,Alaska,R+,9.0,R+,15.0,R+,12.0,R+,18.0
4,Arizona,R+,7.0,R+,8.0,R+,4.0,R+,4.0
5,Arkansas,R+,14.0,R+,14.0,R+,4.0,R+,3.0
6,California,D+,10.0,D+,9.0,D+,6.0,D+,6.0
7,Colorado,D+,1.0,D+,1.0,R+,1.0,R+,5.0
8,Connecticut,D+,7.0,D+,8.0,D+,7.0,D+,9.0
9,Delaware,D+,7.0,D+,9.0,D+,5.0,D+,6.0
10,Florida,R+,2.0,R+,2.0,R+,1.0,R+,0.0
11,Georgia,R+,6.0,R+,6.0,R+,7.0,R+,6.0


In [17]:
def signed_pvi(col1val,col2val):
    if col1val=='D+' and col2val!=0:
        return -col2val
    else:
        return col2val
    
for year in ['2012','2008','2004','2000']: # hell yeah!!!!
    pvis[year] = pvis.apply(lambda x: signed_pvi(x[year + ' PVI direction'],x[year + ' PVI magnitude']),axis=1)
pvis.index = [state.upper() for state in pvis['State']] 

In [18]:
pvis = pvis.iloc[:-1,9:]
pvis

,2012,2008,2004,2000
ALABAMA,13.0,15.0,12.0,8.0
ALASKA,9.0,15.0,12.0,18.0
ARIZONA,7.0,8.0,4.0,4.0
ARKANSAS,14.0,14.0,4.0,3.0
CALIFORNIA,-10.0,-9.0,-6.0,-6.0
COLORADO,-1.0,-1.0,1.0,5.0
CONNECTICUT,-7.0,-8.0,-7.0,-9.0
DELAWARE,-7.0,-9.0,-5.0,-6.0
FLORIDA,2.0,2.0,1.0,0.0
GEORGIA,6.0,6.0,7.0,6.0


In [19]:
# now add pvis to dataset
def get_pvi(state,year):
    if year%4!=0: # pvis are only calculate once every 4 years, but i have midterm elections in my data set
        year-=2 # use pvi for most recent presidential election year
    year = str(year)
    return pvis.loc[state,year]

elections['state enviro'] = elections.apply(lambda x: get_pvi(x.state,x.year), axis=1)

In [29]:
elections

,year,state_po,Dem_candidate,Repub_candidate,Repub%of_two-party_vote,state,Dem_cfscore,Repub_cfscore,state enviro
7,2000,CA,DIANNE FEINSTEIN,TOM CAMPBELL,0.395835,CALIFORNIA,-0.949979,0.570457,-6.0
13,2000,CT,JOSEPH I. LIEBERMAN,PHIL GIORDANO,0.350888,CONNECTICUT,-0.540546,0.159063,-9.0
18,2000,DE,THOMAS R. CARPER,"WILLIAM V. ROTH, JR.",0.4404,DELAWARE,-0.600173,0.644361,-6.0
22,2000,FL,BILL NELSON,BILL MCCOLLUM,0.475053,FLORIDA,-0.619222,0.863845,0.0
38,2000,HI,DANIEL K. AKAKA,JOHN CARROLL,0.252149,HAWAII,-0.781409,0.684819,-10.0
...,...,...,...,...,...,...,...,...,...
1211,2012,VA,TIMOTHY M. KAINE,GEORGE F. ALLEN,0.470423,VIRGINIA,-1.084158,0.966839,0.0
1213,2012,WA,MARIA CANTWELL,MICHAEL BAUMGARTNER,0.39549,WASHINGTON,-1.138804,1.171569,-6.0
1216,2012,WV,JOE MANCHIN III,JOHN R. RAESE,0.375827,WEST VIRGINIA,-0.189099,1.034782,16.0
1221,2012,WI,TAMMY BALDWIN,TOMMY G. THOMPSON,0.471478,WISCONSIN,-1.606083,1.016576,-2.0


<font size=10>adding natl enviro and economy</font>

In [20]:
# set this up to allow for adding each state's unemployment later
info_dict = {
    2000: {'natl enviro': 0.5, # this is actual house pop vote margin, not polls. From wikipedia
           'inflation': 3.38,
           'natl unemploy': 3.99,
          },
    2002: {'natl enviro': 4.8, 
           'inflation': 1.59,
           'natl unemploy': 5.78,
          },
    2004: {'natl enviro': 2.6,
           'inflation': 2.68,
           'natl unemploy': 5.53,
          },
    2006: {'natl enviro': -8.0, 
           'inflation': 3.23,
           'natl unemploy': 4.62,
          },
    2008: {'natl enviro': -10.6,
           'inflation': 3.84,
           'natl unemploy': 5.78,
          },
    2010: {'natl enviro': 6.8, 
           'inflation': 1.64,
           'natl unemploy': 9.63,
          },
    2012: {'natl enviro': -1.2,
           'inflation': 2.07,
           'natl unemploy': 8.07,
          }
}

In [21]:
elections['Natl_enviro'] = elections.apply(lambda x: info_dict[x.year]['natl enviro'], axis=1)

In [32]:
elections

,year,state_po,Dem_candidate,Repub_candidate,Repub%of_two-party_vote,state,Dem_cfscore,Repub_cfscore,state enviro,Natl_enviro
7,2000,CA,DIANNE FEINSTEIN,TOM CAMPBELL,0.395835,CALIFORNIA,-0.949979,0.570457,-6.0,0.5
13,2000,CT,JOSEPH I. LIEBERMAN,PHIL GIORDANO,0.350888,CONNECTICUT,-0.540546,0.159063,-9.0,0.5
18,2000,DE,THOMAS R. CARPER,"WILLIAM V. ROTH, JR.",0.4404,DELAWARE,-0.600173,0.644361,-6.0,0.5
22,2000,FL,BILL NELSON,BILL MCCOLLUM,0.475053,FLORIDA,-0.619222,0.863845,0.0,0.5
38,2000,HI,DANIEL K. AKAKA,JOHN CARROLL,0.252149,HAWAII,-0.781409,0.684819,-10.0,0.5
...,...,...,...,...,...,...,...,...,...,...
1211,2012,VA,TIMOTHY M. KAINE,GEORGE F. ALLEN,0.470423,VIRGINIA,-1.084158,0.966839,0.0,-1.2
1213,2012,WA,MARIA CANTWELL,MICHAEL BAUMGARTNER,0.39549,WASHINGTON,-1.138804,1.171569,-6.0,-1.2
1216,2012,WV,JOE MANCHIN III,JOHN R. RAESE,0.375827,WEST VIRGINIA,-0.189099,1.034782,16.0,-1.2
1221,2012,WI,TAMMY BALDWIN,TOMMY G. THOMPSON,0.471478,WISCONSIN,-1.606083,1.016576,-2.0,-1.2


In [22]:
elections['Inflation'] = elections.apply(lambda x: info_dict[x.year]['inflation'], axis=1)
elections['Natl_unempl'] = elections.apply(lambda x: info_dict[x.year]['natl unemploy'], axis=1)

In [23]:
elections

,year,state_po,Dem_candidate,Repub_candidate,Repub%of_two-party_vote,state,Dem_cfscore,Repub_cfscore,state enviro,Natl_enviro,Inflation,Natl_unempl
7,2000,CA,DIANNE FEINSTEIN,TOM CAMPBELL,0.395835,CALIFORNIA,-0.949979,0.570457,-6.0,0.5,3.38,3.99
13,2000,CT,JOSEPH I. LIEBERMAN,PHIL GIORDANO,0.350888,CONNECTICUT,-0.540546,0.159063,-9.0,0.5,3.38,3.99
18,2000,DE,THOMAS R. CARPER,"WILLIAM V. ROTH, JR.",0.4404,DELAWARE,-0.600173,0.644361,-6.0,0.5,3.38,3.99
22,2000,FL,BILL NELSON,BILL MCCOLLUM,0.475053,FLORIDA,-0.619222,0.863845,0.0,0.5,3.38,3.99
38,2000,HI,DANIEL K. AKAKA,JOHN CARROLL,0.252149,HAWAII,-0.781409,0.684819,-10.0,0.5,3.38,3.99
...,...,...,...,...,...,...,...,...,...,...,...,...
1211,2012,VA,TIMOTHY M. KAINE,GEORGE F. ALLEN,0.470423,VIRGINIA,-1.084158,0.966839,0.0,-1.2,2.07,8.07
1213,2012,WA,MARIA CANTWELL,MICHAEL BAUMGARTNER,0.39549,WASHINGTON,-1.138804,1.171569,-6.0,-1.2,2.07,8.07
1216,2012,WV,JOE MANCHIN III,JOHN R. RAESE,0.375827,WEST VIRGINIA,-0.189099,1.034782,16.0,-1.2,2.07,8.07
1221,2012,WI,TAMMY BALDWIN,TOMMY G. THOMPSON,0.471478,WISCONSIN,-1.606083,1.016576,-2.0,-1.2,2.07,8.07


In [35]:
# saving the dataset to csv so can load in other notebooks
elections.to_csv('..\\prepared_data\\dataset_7-24.csv',index=False)

<font size = 6>Data Preparation</font>

<font size = 4>Splitting into training and validation sets, and handling missing values and categorical variables</font>

My only categorical data is year, state, and the candidate's name itself. Yhe lattermost seems unimportant, but i expect that keeping the state itself, beyond just its PVI, is informative because of things like some states being more idiosyncratic than others. As for year, I can't use that as a variable because models cannot predict for categories they were never trained on, so I would not be able to perform inference on future years.

In [29]:
dataset = pd.read_csv('..\\prepared_data\\dataset_7-24.csv') # or whatever version of dataset most recent
dataset

,year,state_po,Dem_candidate,Repub_candidate,Repub%of_two-party_vote,state,Dem_cfscore,Repub_cfscore,state enviro,Natl_enviro,Inflation,Natl_unempl
0,2000,CA,DIANNE FEINSTEIN,TOM CAMPBELL,0.395835,CALIFORNIA,-0.949979,0.570457,-6.0,0.5,3.38,3.99
1,2000,CT,JOSEPH I. LIEBERMAN,PHIL GIORDANO,0.350888,CONNECTICUT,-0.540546,0.159063,-9.0,0.5,3.38,3.99
2,2000,DE,THOMAS R. CARPER,"WILLIAM V. ROTH, JR.",0.440400,DELAWARE,-0.600173,0.644361,-6.0,0.5,3.38,3.99
3,2000,FL,BILL NELSON,BILL MCCOLLUM,0.475053,FLORIDA,-0.619222,0.863845,0.0,0.5,3.38,3.99
4,2000,HI,DANIEL K. AKAKA,JOHN CARROLL,0.252149,HAWAII,-0.781409,0.684819,-10.0,0.5,3.38,3.99
...,...,...,...,...,...,...,...,...,...,...,...,...
211,2012,VA,TIMOTHY M. KAINE,GEORGE F. ALLEN,0.470423,VIRGINIA,-1.084158,0.966839,0.0,-1.2,2.07,8.07
212,2012,WA,MARIA CANTWELL,MICHAEL BAUMGARTNER,0.395490,WASHINGTON,-1.138804,1.171569,-6.0,-1.2,2.07,8.07
213,2012,WV,JOE MANCHIN III,JOHN R. RAESE,0.375827,WEST VIRGINIA,-0.189099,1.034782,16.0,-1.2,2.07,8.07
214,2012,WI,TAMMY BALDWIN,TOMMY G. THOMPSON,0.471478,WISCONSIN,-1.606083,1.016576,-2.0,-1.2,2.07,8.07


In [30]:
dataset = dataset.drop(columns=['Dem_candidate','Repub_candidate','state']) 

In [31]:
dep_var='Repub%of_two-party_vote'
procs = [Categorify, FillMissing]

cond = dataset.year<=2010
import numpy as np
train_idx = np.where(cond)[0] 
valid_idx = np.where(~cond)[0] 
splits = (list(train_idx),list(valid_idx))

dataset.drop(columns='year',inplace=True) # we were only using years to split into train and valid sets

# tell TabularPandas which columns are continuous vs categ
cont, cat = cont_cat_split(dataset, 50, dep_var='Repub%of_two-party_vote') # splits into cont or cat based on whether var has more than whatever the specified number (e.g., 50) of different values

In [32]:
to = TabularPandas(dataset, procs, cat, cont, y_names='Repub%of_two-party_vote', splits=splits)

In [33]:
to.classes

{'state_po': ['#na#', 'AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY'],
 'Dem_cfscore_na': ['#na#', False, True],
 'Repub_cfscore_na': ['#na#', False, True]}

In [40]:
save_pickle('..\\prepared_data\\to.pkl',to)